# **Régularisation**

### Import des librairies

In [ ]:
%tensorflow_version 2.x
from sklearn.preprocessing import normalize
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import fashion_mnist

## Télécharger un modèle pré entrainé comme InceptionV3 (exemple)

###  En utilisant [ce lien](https://keras.io/applications)

Dans cette séance nous n'allons pas utiliser de modèle téléchargé car ils sont généralement assez coûteux à entrainer, mais l'exemple ci-dessous vous montre qu'il est très facile de récupérer un modèle pré-entrainé.



In [ ]:
model = keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

### Afficher les informations de ce modèle

In [ ]:
model.summary()

### Charger les données MNIST

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# réduire la taille du train pour accélérer les experiences
nnn=1024
x_train = x_train[:nnn]
y_train = y_train[:nnn]

print("x_train",x_train.shape)
print("Nombre d'images pour entrainer:",x_train.shape[0])
print("Nombre d'images pour tester:",x_test.shape[0])
print("La taille d'une image:",x_train.shape[1:],"ce qui fait au total:",
      x_train.shape[1]*x_train.shape[2],"pixels")
print("La liste des classes:",np.unique(y_train))

### Définition d'une fonction qui transforme en représentation binaire pour plusieurs classes

In [ ]:
def transform_labels(y_train,y_test):
  """
  Cette fonction transform les classes non-binaire à une représentation binaire
  Par exemple si on a une liste de 6 fleures chacune peut avoir une des 3 classes
  Entrée: [1,3,3,2,1,2]
  Sortie: [
           [1,0,0], # class 1
           [0,0,1], # class 3
           [0,0,1], # class 3
           [0,1,0], # class 2
           [1,0,0], # class 1
           [0,1,0]  # class 2
          ]
  """

  # concatener train et test
  y_train_test = np.concatenate((y_train,y_test),axis =0)

  # init un encoder Label
  encoder = LabelEncoder()
  # transformer de [1,3,3,2,1,2] à [0,2,2,1,0,1]
  new_y_train_test = encoder.fit_transform(y_train_test)

  # init un encoder one-hot
  encoder = OneHotEncoder()
  # préparer la transformation de [0,2,2,1,0,1] à la représentation binaire
  encoder.fit(new_y_train_test.reshape(-1,1))

  # resplit the train and test
  new_y_train = new_y_train_test[0:len(y_train)]
  new_y_test = new_y_train_test[len(y_train):]

  return new_y_train, new_y_test, encoder

### Visualiser quelques images en spécifiant que c'est du noir et blanc

In [ ]:
plt.subplot(221)
plt.imshow(x_train[0], cmap=plt.get_cmap('gray'))
plt.subplot(222)
plt.imshow(x_train[1], cmap=plt.get_cmap('gray'))
plt.subplot(223)
plt.imshow(x_train[2], cmap=plt.get_cmap('gray'))
plt.subplot(224)
plt.imshow(x_train[3], cmap=plt.get_cmap('gray'))
# show the plot
plt.show()

### Normaliser les données pour que chaque pixel soit entre 0 et 1

In [ ]:
# diviser par 255 pour que tout soit entre 0 et 1
x_train = x_train/255
x_test = x_test/255

## Augmentation de données

### Définir un générateur qui prend en entrée un jeu de données et retourne son augmentation

In [ ]:
# utiliser un datagen de keras
datagen = keras.preprocessing.image.ImageDataGenerator(featurewise_center=True,
                                                       featurewise_std_normalization=True,
                                                       rotation_range=20, # angle de rotation aléatoire
                                                       width_shift_range=0.1, #  décalage horizontale
                                                       height_shift_range = 0.1, # décalage vertical
                                                       brightness_range = (0.0,1.0) # changement de l'eclairage
                                                      )

### Ajouter une dimension vide pour remplacer le RGB

In [ ]:
x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]

### Tester l'augmentation sur 9 images pour voir

In [ ]:
for i in range(0, 9):
	plt.subplot(330 + 1 + i)
	plt.imshow(x_train[i,:,:,0], cmap=plt.get_cmap('gray'))
# show the plot
plt.show()

# donner les données à augmenter
datagen.fit(x_train,augment=True)

# itérer sur une augmentation en ligne
for x_batch, y_batch in datagen.flow(x_train,y_train,batch_size = 9,shuffle=False):
  # tracer celle qui a subi une modification aléatorie
  for i in range(9):
    plt.subplot(330+1+i)
    plt.imshow(x_batch[i,:,:,0], cmap=plt.get_cmap('gray'))
  plt.show()
  plt.close()
  break

### Transformer les classes en représentation binaire (one-hot encoding)

In [ ]:
y_train_binaire,y_test_binaire,encoder = transform_labels(y_train,y_test)

### Créer un modèle convolutif à deux couches avec la sortie softmax classifier

In [ ]:
# créer la couche  d'entrée
input_shape = x_train.shape[1:]
input_layer = keras.layers.Input(input_shape)

# créer la couche convolutive
hidden_conv_layer_1 = keras.layers.Conv2D(filters=16,
                                          kernel_size=(5,5) ,strides=1,
                                          padding='valid',activation='relu')(input_layer)
# créer la deuxième couche convolutive
hidden_conv_layer_2 = keras.layers.Conv2D(filters=8,
                                          kernel_size=(5,5) ,strides=1,
                                          padding='valid')(hidden_conv_layer_1)

# ajouter une couche de batch normalization avant l'activation
batch_norm_layer_2 = keras.layers.BatchNormalization(axis=-1)(hidden_conv_layer_2)

# ajouter l'activation relu
activation_layer_2 = keras.layers.Activation('relu')(batch_norm_layer_2)

# aplatir l'image
flatenned_layer_2 = keras.layers.Flatten()(activation_layer_2)

nb_classes= len(np.unique(y_train))

# créer la couche de sortie
output_layer = keras.layers.Dense(units=nb_classes,activation='softmax')(flatenned_layer_2)

# créer le modèle en liant l'input à l'output
model = keras.models.Model(inputs=input_layer, outputs=output_layer)

# visualiser le modèle construit
model.summary()

### Entrainer le modèle avec l'interface de data augmentation

In [ ]:
# choisir l'algorithme d'optimization
optimizer_algo = keras.optimizers.SGD(lr=0.1)

# fonction de coût qu'on veut optimiser
cost_function = keras.losses.categorical_crossentropy

# compiler le modèle
model.compile(loss=cost_function,optimizer=optimizer_algo, metrics=['accuracy'])

# mini batch size et le nombre d'epoque
mini_batch_size = 128
nb_epochs = 100

# entrainer l'augmenteur d'images
datagen.fit(x_train,augment=True)

# entrainer en utilisant l'interface d'augmentation de données
for epoch in range(nb_epochs):
  batches = 0
  for x_batch,y_batch in datagen.flow(x_train,y_train,batch_size=mini_batch_size):
    y_batch_binaire = encoder.transform(y_batch[:,np.newaxis])
    y_batch_binaire = y_batch_binaire.toarray()
    model.fit(x_batch,y_batch_binaire,verbose=True)
    batches += 1
    if batches > len(x_train) / mini_batch_size:
      break

### Evaluer sur le test de MNIST

In [ ]:
y_test_binaire_now = encoder.transform(y_test_binaire[...,np.newaxis])

y_test_binaire_now = y_test_binaire_now.toarray()

loss,acc = model.evaluate(x_test,y_test_binaire_now,verbose=False)

print("L'accuracy sur l'ensemble de test est :",acc)

## Transfer learning entre le dataset Fashion_MNIST et MNIST

### Télécharger Fashion_MNIST  

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train[:1000]
y_train = y_train[:1000]

### Afficher des exemples du nouveau jeu de données

In [ ]:
plt.subplot(221)
plt.imshow(x_train[0], cmap=plt.get_cmap('gray'))
plt.subplot(222)
plt.imshow(x_train[1], cmap=plt.get_cmap('gray'))
plt.subplot(223)
plt.imshow(x_train[2], cmap=plt.get_cmap('gray'))
plt.subplot(224)
plt.imshow(x_train[3], cmap=plt.get_cmap('gray'))
# show the plot
plt.show()

### Normaliser les données pour que chaque pixel soit entre 0 et 1

In [ ]:
# diviser par 255 pour que tout soit entre 0 et 1
x_train = x_train/255
x_test = x_test/255

## Transformer en classes one-hot encoding

In [ ]:
y_train_binaire,y_test_binaire,encoder = transform_labels(y_train,y_test)

y_train_binaire = encoder.transform(y_train_binaire[...,np.newaxis])
y_test_binaire = encoder.transform(y_test_binaire[...,np.newaxis])

### Ajouter une dimension vide pour remplacer le RGB

In [ ]:
x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]

### Évaluer sur Fashion_MNIST la performance du modèle déjà entrainé sur MNIST

In [ ]:
if type(y_test_binaire) != np.ndarray:
  y_test_binaire = y_test_binaire.toarray()

loss,acc = model.evaluate(x_test,y_test_binaire,verbose=False)

print("L'accuracy sur l'ensemble du test est :",acc)

### Re-entrainer le modèle sur l'ensemble du train de Fashion_MNIST

In [ ]:
y_train_binaire = y_train_binaire.toarray()

history = model.fit(x_train,y_train_binaire,batch_size = 256, epochs=100, validation_split = 0.3, verbose=False)

### Tracer la variation de l'accuracy sur le train et sur le validation set en fonction du nombre d'epoque

In [ ]:
history_dict = history.history
loss_train_epochs = history_dict['accuracy']
loss_val_epochs = history_dict['val_accuracy']

plt.figure()
plt.plot(loss_train_epochs,color='blue',label='train_acc')
plt.plot(loss_val_epochs,color='red',label='val_acc')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.savefig('epoch-loss.pdf')
plt.show()
plt.close()

## **Exercices**

Entrainer un modèle (avec l'architecture leNet-5) sur 1000 images dans FASHION_MNIST et le re-entrainer sur MNIST (si c'est possible ajouter une couche [Dropout](https://keras.io/layers/core/#dropout) avec un rate de 0.3)

### Construire un modèle qui marche assez bien sur Fashion_Mnist

In [ ]:
keras.backend.clear_session()
# télécharger les données fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# réduire la taille du train pour accélérer les experiences
nnn=1024
x_train = x_train[:nnn]
y_train = y_train[:nnn]

# diviser par 255 pour que tout soit entre 0 et 1
x_train = x_train/255
x_test = x_test/255

# one-hot encoding
y_train,y_test,encoder = transform_labels(y_train,y_test)
y_train_binaire = encoder.transform(y_train[...,np.newaxis]).toarray()
y_test_binaire = encoder.transform(y_test[...,np.newaxis]).toarray()

# ajouter une dimension pour remplacer le RGB
x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]

# créer le modèle
# créer la couche d'entrainement
input_shape = x_train.shape[1:]
input_layer = keras.layers.Input(input_shape)
# créer la première covolution
conv_1 = keras.layers.Conv2D(filters=6,kernel_size=(5,5),padding='same',activation='sigmoid')(input_layer)
# créer le subsampling ou bien le pooling
pool_1 = keras.layers.MaxPooling2D(pool_size=2)(conv_1)
# ajouter un dropout de 0.3
drop_1 = keras.layers.Dropout(0.3)(pool_1)

# créer la deuxième convolution
conv_2 = keras.layers.Conv2D(filters=16,kernel_size=(5,5),padding='valid',activation='sigmoid')(drop_1)
# créer le pooling 2
pool_2 = keras.layers.MaxPooling2D(pool_size=2)(conv_2)
# ajouter un dropout de 0.3
drop_2 = keras.layers.Dropout(0.3)(pool_2)

# aplatir l'image
flattened_layer_2 = keras.layers.Flatten()(drop_2)

# un fully connected ou bien un Dense layer
layer_3 = keras.layers.Dense(units=120, activation='sigmoid')(flattened_layer_2)
# ajouter un dropout de 0.3
drop_3 = keras.layers.Dropout(0.3)(layer_3)

# un fully connected ou bien un Dense layer
layer_4 = keras.layers.Dense(units=84, activation='sigmoid')(drop_3)
# ajouter un dropout de 0.3
drop_4 = keras.layers.Dropout(0.3)(layer_4)

# la dernière couche qui a un nombre de neuroe égale au nombre de classes
nb_classes= y_train_binaire.shape[1]
output_layer = keras.layers.Dense(units=nb_classes,activation='softmax')(drop_4)

# construire le modèle
model = keras.models.Model(inputs=input_layer, outputs=output_layer)

# afficher les informations
model.summary()

# entrainer

# choisir le taux d'apprentissage pour la descente du gradient
optimizer_algo = keras.optimizers.Adam(lr=0.01)

# fonction de cout
cost_function = keras.losses.categorical_crossentropy

# compiler le modèle
model.compile(loss=cost_function,optimizer=optimizer_algo, metrics=['accuracy'])

# entrainer
history = model.fit(x_train,y_train_binaire,batch_size=256,epochs=100,verbose=False,
                   validation_split=0.3)

# evaluer sur le test
loss,acc = model.evaluate(x_test,y_test_binaire,verbose=False)
print("L'accuracy sur l'ensemble du test est:",acc)

# tracer l'evaluation de l'accuracy
history_dict = history.history
loss_train_epochs = history_dict['accuracy']
loss_val_epochs = history_dict['val_accuracy']

plt.figure()
plt.plot(loss_train_epochs,color='blue',label='train_acc')
plt.plot(loss_val_epochs,color='red',label='val_acc')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.savefig('epoch-loss.pdf')
plt.show()
plt.close()

### Télécharger les données MNIST et re-entrainer le modèle pour 100 époques et visualiser l'history (n'oubilier pas le pré-traitement des données)

In [ ]:
# télécharger les données mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# réduire la taille du train pour accélérer les experiences
nnn=1024
x_train = x_train[:nnn]
y_train = y_train[:nnn]

# diviser par 255 pour que tout soit entre 0 et 1
x_train = x_train/255
x_test = x_test / 255

# one-hot encoding
y_train,y_test,encoder = transform_labels(y_train,y_test)
y_train_binaire = encoder.transform(y_train[...,np.newaxis]).toarray()
y_test_binaire = encoder.transform(y_test[...,np.newaxis]).toarray()

# ajouter une dimension pour remplacer le RGB
x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]

# entrainer
history = model.fit(x_train,y_train_binaire,batch_size=256,epochs=100,verbose=False,
                   validation_split=0.3)

# evaluer sur le test
loss,acc = model.evaluate(x_test,y_test_binaire,verbose=False)
print("L'accuracy sur l'ensemble du test est:",acc)

# tracer l'evaluation de l'accuracy
history_dict = history.history
loss_train_epochs = history_dict['accuracy']
loss_val_epochs = history_dict['val_accuracy']

plt.figure()
plt.plot(loss_train_epochs,color='blue',label='train_acc')
plt.plot(loss_val_epochs,color='red',label='val_acc')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.savefig('epoch-loss.pdf')
plt.show()
plt.close()